In [3]:
import pandas as pd

In [5]:
data = pd.read_csv("datanep.csv")
delta = data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI'] = 100 - (100 / (1 + rs))
data['RSI'].fillna(0, inplace=True)
data['delta']= data['Close'].diff()
print(data[1110:])

            Date    Close Percent Change             Volume        RSI  delta
1110  11/10/2024  2738.06          2.33%   9,771,124,073.55  55.078290  62.55
1111  11/11/2024  2760.21          0.80%  11,315,207,127.09  52.427103  22.15
1112  11/12/2024  2748.96         -0.40%   7,859,740,798.91  50.492099 -11.25
1113  11/13/2024  2734.83         -0.51%   7,911,976,449.08  55.552548 -14.13
1114  11/14/2024  2748.78          0.51%  10,320,331,436.20  68.871784  13.95
1115  11/17/2024  2753.76          0.18%  11,726,756,904.61  66.438638   4.98
1116  11/19/2024  2702.65         -1.85%  10,468,320,521.90  59.600939 -51.11
1117  11/20/2024  2711.75          0.33%   7,092,879,946.39  64.635876   9.10
1118  11/21/2024  2680.44         -1.15%   8,257,643,886.83  60.041135 -31.31
1119  11/24/2024  2667.40         -0.48%   7,728,861,250.73  53.712938 -13.04
1120  11/25/2024  2683.24          0.59%   6,345,809,597.84  48.343674  15.84
1121  11/26/2024  2716.66          1.24%   7,449,898,987.89  56.

C:\Users\Asus\AppData\Local\Temp\ipykernel_4664\867565060.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['RSI'].fillna(0, inplace=True)


MACD calculation



In [ ]:
data['ema12']= data['close'].ewm(span= 12, adjust= False).mean()

data['ema26'] = data['close'].ewm(span = 26, adjust = False).mean()

